In [1]:
import tensorflow as tf
import numpy as np 
from matplotlib import pyplot as plt
from keras.datasets import mnist
from random import seed
from random import randrange
from random import random
from random import uniform
from csv import reader
from math import exp

In [2]:
#Assignment ONE
(x_train,y_train),(x_test,y_test)= mnist.load_data()
#x_train and x_test selected from last sample
x_train=x_train[-15000:-5000]
y_train=y_train[-15000:-5000]

x_test=x_test[-1000:]
y_test=y_test[-1000:]



In [3]:
def get_ImBlock(image , row , col ):
    x , y = image.shape
    return (image.reshape ( x //row, row, -1, col) .swapaxes(1,2).reshape(-1, row, col))

In [4]:
def get_centroid(img):
    feature_vector = []
    for block in get_ImBlock(img , 4 , 4 ) :
        
        Xc = 0 
        Yc = 0 
        sum = 0
    
        for index, x in np.ndenumerate(block):
          sum+= x 
          Xc += x * index[0]
          Yc += x * index[1]
        
        if sum != 0 :
            feature_vector.append( Xc/ sum )
            feature_vector.append(Yc/ sum )
        else :
             feature_vector.append(0)
             feature_vector.append(0)
        
    
    return np.array(feature_vector)

In [5]:
# Initialize a network
def initialize_network(n_i, n_j, n_k):
    network = list()
    hidden_layer = [{'weights':[uniform (-1,1)for i in range(n_i + 1)]} for i in range(n_j)]
    network.append(hidden_layer)
    output_layer = [{'weights':[uniform (-1,1)for i in range(n_j + 1)]} for i in range(n_k)]
    network.append(output_layer)
    return network


In [6]:
# Calculate Sum(weights*input)
def Calculate_Sum(weights, inputs):
    weight=np.array(weights)
    inputs=np.array(inputs)
    inputs=np.append(inputs, np.ones(1).reshape((1,))) #add columns of 1
    ##############################################################################
    inputs=np.transpose(inputs)
    Net=np.dot(weight,inputs)
    return Net



In [7]:
# activation function using sigmoid
def sigmoid(Z):
    return 1.0 / (1.0 + exp(-Z))


In [8]:
# Forward propagate
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            Netj = Calculate_Sum(neuron['weights'], inputs)
            neuron['output'] = sigmoid(Netj)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [9]:
# Calculate the derivative of an neuron output: output * (1.0 - output)
def transfer_derivative(output):
    return output * (1.0 - output)

In [10]:
#Backpropagate error and store in neurons
#Output layer: error = (O - T) * transfer_derivative(O)
#Hidden layer: error = (weight_k * error_j) * transfer_derivative(O)
def backward_propagate_error(network, target):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output'] - target[j])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [11]:
#This function update the weights
#weight = weight - learning_rate * error * input
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']

In [12]:
#find max
def maxValue(array):
    max=array[0]
    indexMax=0
    for i in range(len(array)):
        if max < array[i]:
            max=array[i]
            indexMax=i
    return indexMax

In [13]:
# Train a network for a fixed number of epochs
def train_network(network, x_train,y_train, l_rate, n_epoch):
    for epoch in range(n_epoch):
        sum_error = 0
        j=0
        for row in x_train:
            outputs = forward_propagate(network, row)
            target=y_train[j]
            j+=1
            sum_error += sum([(target[i]-outputs[i])**2 for i in range(len(target))])
            backward_propagate_error(network, target)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [14]:
#get new features for train
train_features = [get_centroid(img)  for img in x_train ]
train_features = np.array(train_features)
train_features.shape
#transform Y_train to one hot vector
dimensions=(len(y_train),10)
new_Y_train=np.zeros(dimensions)
row=0
for i in y_train:
    new_Y_train[row][i]=1
    row+=1
print(new_Y_train)
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
[3 1 1 ... 0 4 0]


In [15]:
#get new features for test
test_features = [get_centroid(img)  for img in x_test ]
test_features = np.array(test_features)

#transform Y_test to one hot vector
dimensions=(len(y_train),10)
new_Y_test=np.zeros(dimensions)
row=0
for i in y_test:
    new_Y_test[row][i]=1
    row+=1
print(new_Y_test)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
#initialize network
transposed_xtrain=train_features.transpose()
transposed_ytrain=new_Y_train.transpose()
N_input=len(transposed_xtrain)
N_j=19
N_output=len(transposed_ytrain)
network = initialize_network(N_input, N_j, N_output)
print(N_input)
print(N_output)
i=0   

98
10


In [17]:
#forward propagate test
inputs = train_features[0]
forward_propagate(network, inputs)

[0.09720464742983821,
 0.2778584352143303,
 0.31711245749008865,
 0.566458082192888,
 0.15544965171123207,
 0.8941309262765021,
 0.6794393918343149,
 0.6987149998947014,
 0.01351472294451912,
 0.7090878008366094]

In [18]:
#back propagate test
backward_propagate_error(network, new_Y_train)

In [19]:
#Train the NN to the MNIST dataset
l_rate=0.05
n_epoch=70
x_train=train_features
y_train=new_Y_train
train_network(network, x_train,y_train, l_rate, n_epoch)

>epoch=0, lrate=0.050, error=6700.771
>epoch=1, lrate=0.050, error=4156.280
>epoch=2, lrate=0.050, error=3305.280
>epoch=3, lrate=0.050, error=2902.368
>epoch=4, lrate=0.050, error=2668.194
>epoch=5, lrate=0.050, error=2510.739
>epoch=6, lrate=0.050, error=2394.615
>epoch=7, lrate=0.050, error=2302.180
>epoch=8, lrate=0.050, error=2224.055
>epoch=9, lrate=0.050, error=2156.236
>epoch=10, lrate=0.050, error=2096.949
>epoch=11, lrate=0.050, error=2045.280
>epoch=12, lrate=0.050, error=1999.893
>epoch=13, lrate=0.050, error=1959.483
>epoch=14, lrate=0.050, error=1922.987
>epoch=15, lrate=0.050, error=1889.529
>epoch=16, lrate=0.050, error=1858.398
>epoch=17, lrate=0.050, error=1829.030
>epoch=18, lrate=0.050, error=1801.285
>epoch=19, lrate=0.050, error=1775.240
>epoch=20, lrate=0.050, error=1750.905
>epoch=21, lrate=0.050, error=1728.182
>epoch=22, lrate=0.050, error=1706.878
>epoch=23, lrate=0.050, error=1686.833
>epoch=24, lrate=0.050, error=1667.955
>epoch=25, lrate=0.050, error=1650.

In [20]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return maxValue(outputs)

In [21]:
#test
i=0
for row in test_features:
    prediction = predict(network, row)
    print('target=%d, Got=%d' % (y_test[i], prediction))
    i+=1

target=7, Got=7
target=6, Got=6
target=1, Got=1
target=1, Got=1
target=0, Got=0
target=1, Got=1
target=2, Got=8
target=3, Got=3
target=4, Got=8
target=7, Got=2
target=2, Got=8
target=3, Got=3
target=4, Got=4
target=5, Got=4
target=6, Got=6
target=7, Got=8
target=0, Got=0
target=1, Got=1
target=2, Got=2
target=7, Got=7
target=8, Got=8
target=6, Got=6
target=3, Got=3
target=9, Got=9
target=7, Got=3
target=1, Got=8
target=9, Got=9
target=3, Got=3
target=9, Got=9
target=6, Got=6
target=1, Got=1
target=7, Got=7
target=2, Got=2
target=4, Got=4
target=4, Got=4
target=5, Got=1
target=7, Got=7
target=0, Got=0
target=0, Got=0
target=1, Got=1
target=6, Got=6
target=6, Got=6
target=8, Got=8
target=2, Got=2
target=7, Got=7
target=7, Got=7
target=2, Got=8
target=4, Got=4
target=2, Got=2
target=1, Got=1
target=6, Got=6
target=1, Got=1
target=0, Got=0
target=6, Got=6
target=9, Got=9
target=8, Got=8
target=3, Got=5
target=9, Got=9
target=6, Got=6
target=3, Got=3
target=0, Got=0
target=1, Got=2
target=2

target=8, Got=8
target=8, Got=8
target=7, Got=7
target=2, Got=2
target=2, Got=2
target=3, Got=3
target=9, Got=9
target=3, Got=3
target=3, Got=3
target=5, Got=5
target=5, Got=5
target=0, Got=0
target=7, Got=7
target=9, Got=4
target=5, Got=5
target=6, Got=6
target=5, Got=5
target=1, Got=1
target=4, Got=4
target=1, Got=1
target=1, Got=1
target=2, Got=2
target=8, Got=8
target=2, Got=2
target=6, Got=6
target=1, Got=1
target=5, Got=3
target=0, Got=0
target=1, Got=1
target=2, Got=2
target=3, Got=3
target=4, Got=4
target=5, Got=5
target=6, Got=0
target=7, Got=7
target=8, Got=8
target=9, Got=9
target=0, Got=0
target=1, Got=2
target=2, Got=2
target=3, Got=5
target=4, Got=4
target=5, Got=5
target=6, Got=6
target=7, Got=7
target=8, Got=8
target=9, Got=4
target=0, Got=0
target=1, Got=1
target=2, Got=2
target=3, Got=3
target=4, Got=4
target=5, Got=5
target=6, Got=6
target=7, Got=7
target=8, Got=8
target=8, Got=8
target=0, Got=0
target=6, Got=6
target=0, Got=0
target=0, Got=3
target=2, Got=2
target=3

In [22]:
def accuracy(y_pred, y):
    return np.average(y_pred == y)
print(accuracy(prediction, y_test)*100)

10.2


In [23]:
n_correct=0
for i in y_test:
    if y_test[i]==prediction:
        n_correct +=1
        
print(n_correct)                               
print((n_correct/y_test.shape[0])*100)                               

119
11.899999999999999
